# Download dataset

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saatvikmtech","key":"e675c36acec415887dcabdbf052957ab"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ait-511-course-project-1-obesity-risk
!unzip ait-511-course-project-1-obesity-risk.zip

  0% 0.00/570k [00:00<?, ?B/s]
100% 570k/570k [00:00<00:00, 939MB/s]
Archive:  ait-511-course-project-1-obesity-risk.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!kaggle datasets download -d aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster
!unzip obesity-or-cvd-risk-classifyregressorcluster.zip

Dataset URL: https://www.kaggle.com/datasets/aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster
License(s): CC-BY-SA-4.0
  0% 0.00/57.5k [00:00<?, ?B/s]
100% 57.5k/57.5k [00:00<00:00, 188MB/s]
Archive:  obesity-or-cvd-risk-classifyregressorcluster.zip
  inflating: ObesityDataSet.csv      


# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler

In [ ]:
original = pd.read_csv("ObesityDataSet.csv")
original

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [ ]:
train = pd.read_csv("train.csv")
train

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,WeightCategory
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15528,15528,Male,18.000000,1.700000,50.000000,no,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,2.000000,Sometimes,Public_Transportation,Insufficient_Weight
15529,15529,Male,18.000000,1.763101,55.523481,yes,yes,2.786008,3.000000,Sometimes,no,1.962646,yes,0.028202,1.561272,Sometimes,Public_Transportation,Insufficient_Weight
15530,15530,Female,19.010211,1.686936,49.660995,no,yes,1.053534,3.452590,Sometimes,no,1.000000,no,2.001230,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
15531,15531,Male,22.777890,1.805445,85.228116,yes,yes,2.000000,2.092179,Sometimes,no,2.452986,no,0.796770,0.000000,Sometimes,Public_Transportation,Overweight_Level_I


In [ ]:
original.rename(columns={'NObeyesdad': 'WeightCategory'}, inplace=True)
train.drop(columns=['id'], inplace=True)
train = pd.concat([train, original]).drop_duplicates()

In [ ]:
test = pd.read_csv("test.csv")
test

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,15533,Female,19.007177,1.772449,137.852618,yes,yes,3.000000,3.000000,Sometimes,no,2.007348,no,1.465931,0.813235,Sometimes,Public_Transportation
1,15534,Female,21.572114,1.698346,75.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.577824,1.865851,Sometimes,Public_Transportation
2,15535,Male,22.285024,1.737453,82.000000,yes,yes,2.000000,2.720642,Sometimes,no,1.830614,no,0.451009,0.000000,Sometimes,Public_Transportation
3,15536,Male,30.916426,1.775580,120.860386,yes,yes,2.712747,3.000000,Sometimes,no,2.144368,no,0.712726,0.100048,Sometimes,Automobile
4,15537,Female,18.000000,1.670000,65.000000,no,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.000000,1.000000,no,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5220,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation
5221,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation
5222,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation
5223,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile


In [ ]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17620 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          17620 non-null  object 
 1   Age                             17620 non-null  float64
 2   Height                          17620 non-null  float64
 3   Weight                          17620 non-null  float64
 4   family_history_with_overweight  17620 non-null  object 
 5   FAVC                            17620 non-null  object 
 6   FCVC                            17620 non-null  float64
 7   NCP                             17620 non-null  float64
 8   CAEC                            17620 non-null  object 
 9   SMOKE                           17620 non-null  object 
 10  CH2O                            17620 non-null  float64
 11  SCC                             17620 non-null  object 
 12  FAF                             17620 

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5225 entries, 0 to 5224
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              5225 non-null   int64  
 1   Gender                          5225 non-null   object 
 2   Age                             5225 non-null   float64
 3   Height                          5225 non-null   float64
 4   Weight                          5225 non-null   float64
 5   family_history_with_overweight  5225 non-null   object 
 6   FAVC                            5225 non-null   object 
 7   FCVC                            5225 non-null   float64
 8   NCP                             5225 non-null   float64
 9   CAEC                            5225 non-null   object 
 10  SMOKE                           5225 non-null   object 
 11  CH2O                            5225 non-null   float64
 12  SCC                             52

<u>**Abbreviated features meaning**</u>

FAVC: Frequent consumption of high caloric food\
FCVC: Frequency of consumption of vegetables\
NCP: Number of main meals\
CAEC: Consumption of food between meals\
SMOKE: Is a smoker\
CH20: Consumption of water daily\
CALC: Consumption of alcohol\
SCC: Calories consumption monitoring\
FAF: Physical activity frequency\
TUE: Time using technology devices\
MTRANS: Transportation used\
WeightCategory: Type of weight based on body mass index (BMI)

In [ ]:
for col in train.select_dtypes(include=['object']).columns:
    print(f"\nColumn: {col}")
    print(train[col].value_counts())


Column: Gender
Gender
Male      8835
Female    8785
Name: count, dtype: int64

Column: family_history_with_overweight
family_history_with_overweight
yes    14418
no      3202
Name: count, dtype: int64

Column: FAVC
FAVC
yes    16028
no      1592
Name: count, dtype: int64

Column: CAEC
CAEC
Sometimes     14887
Frequently     2094
Always          399
no              240
Name: count, dtype: int64

Column: SMOKE
SMOKE
no     17399
yes      221
Name: count, dtype: int64

Column: SCC
SCC
no     17010
yes      610
Name: count, dtype: int64

Column: CALC
CALC
Sometimes     12665
no             4477
Frequently      477
Always            1
Name: count, dtype: int64

Column: MTRANS
MTRANS
Public_Transportation    14028
Automobile                3125
Walking                    395
Motorbike                   41
Bike                        31
Name: count, dtype: int64

Column: WeightCategory
WeightCategory
Obesity_Type_III       3307
Obesity_Type_II        2700
Normal_Weight          2627
Obesity_

In [ ]:
#Dividing Features
num_col = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
ord_col = ['CAEC', 'CALC']
tar_col = ['WeightCategory']
lab_col = []
for col in train.select_dtypes(include=['object']).columns:
  if col not in (ord_col + tar_col):
    lab_col.append(col)

In [ ]:
#Label Encoding
train['MTRANS'] = np.where(train['MTRANS'].isin(['Walking', 'Bike', 'Motorbike']), 0, 1)
test['MTRANS'] = np.where(test['MTRANS'].isin(['Walking', 'Bike', 'Motorbike']), 0, 1)
for col in lab_col:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
le = LabelEncoder()
train['WeightCategory'] = le.fit_transform(train['WeightCategory'])

In [ ]:
#Ordinal Encoding
ord_cat = [['no', 'Sometimes', 'Frequently', 'Always']]
oe = OrdinalEncoder(categories = ord_cat*2)
train[ord_col] = oe.fit_transform(train[ord_col])
test[ord_col] = oe.transform(test[ord_col])

## Scaling Continuous Features

In [ ]:
xtrain = train.drop(columns=['WeightCategory'])
ytrain = train['WeightCategory']

xtest = test.drop(columns=['id'])

In [ ]:
scaler = StandardScaler()
xtrain[num_col] = scaler.fit_transform(xtrain[num_col])
xtest[num_col] = scaler.transform(xtest[num_col])

In [ ]:
xtrain

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,0.097875,-0.002792,-0.227929,1,1,-0.828762,0.322059,1.0,0,1.215278,0,-1.169698,0.591656,1.0,1
1,0,-1.021963,-1.587545,-1.164230,1,1,-0.828762,0.345459,2.0,0,-0.040994,0,0.022396,0.630666,0.0,1
2,0,-1.021963,0.126956,-1.423611,1,1,-1.053590,-1.879659,1.0,0,-0.188445,0,-0.137291,1.747529,0.0,1
3,0,-0.508758,0.118693,1.654731,1,1,1.053180,0.345459,1.0,0,-0.577247,0,0.580132,0.266216,1.0,1
4,1,1.348946,2.421780,0.232371,1,1,0.450326,-1.095437,1.0,0,-0.074149,0,1.176310,0.517453,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,-0.504568,0.118693,1.659805,1,1,1.053180,0.345459,1.0,0,-0.488275,0,0.828572,0.475215,1.0,1
2107,0,-0.329701,0.547194,1.748403,1,1,1.053180,0.345459,1.0,0,-0.032554,0,0.429365,-0.033781,1.0,1
2108,0,-0.235656,0.588194,1.746369,1,1,1.053180,0.345459,1.0,0,0.048167,0,0.516172,0.044179,1.0,1
2109,0,0.083783,0.443798,1.733362,1,1,1.053180,0.345459,1.0,0,1.361321,0,0.188224,-0.055726,1.0,1


In [ ]:
xtest

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,-0.846909,0.817341,1.904378,1,1,1.053180,0.345459,1.0,0,-0.028905,0,0.577829,0.320992,1.0,1
1,0,-0.401106,-0.021492,-0.481074,1,1,-0.828762,0.345459,1.0,0,-0.040994,0,0.711216,2.066325,1.0,1
2,1,-0.277198,0.421193,-0.215402,1,1,-0.828762,-0.045902,1.0,0,-0.319677,0,-0.632053,-1.027425,1.0,1
3,1,1.222996,0.852784,1.259470,1,1,0.512586,0.345459,1.0,0,0.196528,0,-0.320062,-0.861536,1.0,1
4,0,-1.021963,-0.342364,-0.860605,0,1,-0.828762,0.345459,1.0,0,-0.040994,0,0.022396,0.630666,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5220,1,0.218510,0.751426,1.006198,1,1,0.901841,0.345459,1.0,0,0.208771,0,0.416405,-0.701311,1.0,1
5221,1,-1.021963,0.110429,-1.429902,0,1,1.053180,1.746388,2.0,0,-1.686249,0,1.214490,0.630666,1.0,1
5222,1,-0.656791,1.350596,0.679551,1,1,-0.061274,0.345459,1.0,0,-0.040994,0,0.210794,0.959695,0.0,1
5223,1,1.733384,-0.002769,-0.157709,1,1,0.434469,-1.095437,1.0,0,0.197302,0,-1.169698,0.587280,0.0,1


# Models

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from xgboost import XGBClassifier

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## XGBoost

In [ ]:
lrs = np.logspace(np.log10(0.1), np.log10(0.12), 6)

In [ ]:
param = {
    'n_estimators': [200],
    'learning_rate': lrs,
    'max_depth': [6]
}
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)
gridxgb = GridSearchCV(estimator=xgb, param_grid=param, scoring='accuracy', cv=skf, verbose=3, return_train_score=True)
gridxgb.fit(xtrain, ytrain)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END learning_rate=0.1, max_depth=6, n_estimators=200;, score=(train=0.979, test=0.903) total time=   3.2s
[CV 2/5] END learning_rate=0.1, max_depth=6, n_estimators=200;, score=(train=0.978, test=0.913) total time=   3.2s
[CV 3/5] END learning_rate=0.1, max_depth=6, n_estimators=200;, score=(train=0.978, test=0.911) total time=   5.1s
[CV 4/5] END learning_rate=0.1, max_depth=6, n_estimators=200;, score=(train=0.979, test=0.914) total time=   3.2s
[CV 5/5] END learning_rate=0.1, max_depth=6, n_estimators=200;, score=(train=0.978, test=0.911) total time=   7.6s
[CV 1/5] END learning_rate=0.10371372893366482, max_depth=6, n_estimators=200;, score=(train=0.980, test=0.902) total time=   6.5s
[CV 2/5] END learning_rate=0.10371372893366482, max_depth=6, n_estimators=200;, score=(train=0.979, test=0.912) total time=   5.1s
[CV 3/5] END learning_rate=0.10371372893366482, max_depth=6, n_estimators=200;, score=(train=0.979, tes

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'learning_rate': array([0.1       , 0.10371373, 0.10756538, 0.11156006, 0.1157031 ,
       0.12      ]),
                         'max_depth': [6], 'n_estimators': [200]},
             return_train_score=True, scoring='accuracy', verbose=3)

In [ ]:
print(gridxgb.best_score_)
print(gridxgb.best_params_)
bestxgb = gridxgb.best_estimator_
ypredtrain = bestxgb.predict(xtrain)
print(accuracy_score(ytrain, ypredtrain))
print(classification_report(ytrain, ypredtrain))
print(confusion_matrix(ytrain, ypredtrain))

0.9110102156640181
{'learning_rate': np.float64(0.11570310048031525), 'max_depth': 6, 'n_estimators': 200}
0.9785471055618615
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2137
           1       0.97      0.97      0.97      2627
           2       0.98      0.98      0.98      2558
           3       1.00      1.00      1.00      2700
           4       1.00      1.00      1.00      3307
           5       0.96      0.94      0.95      2120
           6       0.95      0.96      0.95      2171

    accuracy                           0.98     17620
   macro avg       0.98      0.98      0.98     17620
weighted avg       0.98      0.98      0.98     17620

[[2105   31    0    0    0    1    0]
 [  36 2561    0    0    0   23    7]
 [   1    1 2506    8    0   10   32]
 [   0    0    0 2700    0    0    0]
 [   0    0    0    0 3306    1    0]
 [   3   55    6    0    0 1984   72]
 [   0    5   34    1    0   51 2080]]


## Submission file

In [ ]:
model = bestxgb

ypredtest = model.predict(xtest)
test_pred_labels = le.inverse_transform(ypredtest)
submission = pd.DataFrame({
    'id': test['id'],
    'WeightCategory': test_pred_labels
})
submission.to_csv('submission_xtuned2.csv', index=False)

In [ ]:
submission

,id,WeightCategory
0,15533,Obesity_Type_III
1,15534,Overweight_Level_I
2,15535,Overweight_Level_II
3,15536,Obesity_Type_II
4,15537,Normal_Weight
...,...,...
5220,20753,Obesity_Type_II
5221,20754,Insufficient_Weight
5222,20755,Obesity_Type_I
5223,20756,Overweight_Level_II
